In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import cv2
import os
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage import data, exposure

In [17]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
def load_images(file_path):
    sample_size = 5000
    images = []
    hogs = []
    hogimg = []
    mags = []
    angles = []
    counter = 0
    for filename in os.listdir(file_path):
        img = cv2.imread(os.path.join(file_path, filename))
        if img is not None:
            dim = (64,128)
            scaled = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(scaled, cv2.COLOR_BGR2GRAY)
            gray = np.float32(gray)/255.0
            images.append(gray)
            feat, vis = hog(gray, visualize=True)
            hogs.append(feat)
            hogimg.append(vis)
            gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=1)
            gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=1)
            mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
            mags.append(mag)
            angles.append(angle)
            counter += 1
        if counter > sample_size - 1:
            break
    return images, mags, angles, hogs, hogimg
images, mags, angles, hogs, hogimg = load_images(file_path)

In [18]:
label_data = pd.read_csv(r'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv')
gender_label = pd.DataFrame(label_data)
gender_label = gender_label[gender_label.columns[2]]
gender_label = gender_label[:5000]
print(gender_label)

0      -1
1      -1
2       1
3      -1
4      -1
       ..
4995    1
4996    1
4997    1
4998    1
4999    1
Name: gender, Length: 5000, dtype: int64


In [19]:
print(images[0])

[[0.9137255  0.9137255  0.9137255  ... 0.60784316 0.8862745  0.9372549 ]
 [0.9137255  0.9137255  0.9137255  ... 0.64705884 0.9098039  0.94509804]
 [0.9137255  0.9137255  0.9137255  ... 0.6784314  0.91764706 0.94509804]
 ...
 [0.46666667 0.34509805 0.32941177 ... 0.25490198 0.22745098 0.19607843]
 [0.3372549  0.45490196 0.3882353  ... 0.2627451  0.2627451  0.26666668]
 [0.49019608 0.6313726  0.28627452 ... 0.2784314  0.2627451  0.26666668]]


In [20]:
print(mags[0])

[[0.         0.         0.         ... 0.517647   0.32941175 0.        ]
 [0.         0.         0.         ... 0.49525234 0.29968584 0.00784314]
 [0.         0.         0.         ... 0.4558813  0.26738656 0.00392157]
 ...
 [0.17647061 0.13814835 0.16381647 ... 0.09180158 0.10441983 0.07058825]
 [0.02352941 0.29077843 0.2546605  ... 0.02480218 0.03551133 0.07058825]
 [0.         0.20392156 0.41960788 ... 0.01568627 0.01176471 0.        ]]


In [21]:
print(angles[0])

[[  0.          0.          0.        ...   0.          0.
    0.       ]
 [  0.          0.          0.        ...   8.19365     6.008198
   90.       ]
 [  0.          0.          0.        ...   8.408336    4.2046294
  270.       ]
 ...
 [270.        173.48099   137.90228   ... 109.98735   124.27966
   90.       ]
 [ 90.         79.90265   189.75217   ... 108.438675   83.660614
   90.       ]
 [  0.        180.        180.        ... 180.        180.
    0.       ]]


In [22]:
print(hogimg[0])

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.24164892e-02
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.66552364e-05 0.00000000e+00 ... 0.00000000e+00
  1.32602511e-03 0.00000000e+00]
 ...
 [0.00000000e+00 2.44993903e-02 0.00000000e+00 ... 0.00000000e+00
  4.77337353e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.45183495e-02 ... 1.02060242e-02
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [23]:
x_train, x_test, y_train, y_test = train_test_split(hogs,gender_label, test_size=0.25)
print(x_train[0])
print(y_train)

[0.27804317 0.00665411 0.00841685 ... 0.15794688 0.00447381 0.        ]
3260    1
4170    1
952     1
1501   -1
3264    1
       ..
4982    1
4121    1
1858    1
3503    1
2992   -1
Name: gender, Length: 3750, dtype: int64


In [26]:
classifier = LinearSVC(max_iter=3000)
classifier.fit(x_train, y_train)

LinearSVC(max_iter=3000)

In [27]:
y_pred = classifier.predict(x_test)
accuracy = metrics.accuracy_score(y_test,y_pred=y_pred)
print(accuracy)

0.4992
